# Chapter 07-03: BERT Fine-Tuning

## 학습 목표
- BERT의 두 가지 사전학습 목표(MLM, NSP)를 이해한다
- HuggingFace `transformers` 라이브러리와 TensorFlow 백엔드를 함께 사용한다
- AutoTokenizer로 텍스트를 토크나이징하는 방법을 익힌다
- `TFAutoModelForSequenceClassification`으로 감성 분류 Fine-Tuning을 수행한다
- 한국어 BERT 모델의 종류와 특징을 파악한다

## 목차
1. 라이브러리 임포트
2. BERT 사전학습 목표 이해
3. HuggingFace + TF 백엔드 사용 방법
4. 모델 및 토크나이저 로드
5. 토크나이저 사용법
6. Fine-Tuning 코드
7. 한국어 BERT 모델 목록
8. 정리

In [ ]:
import os
import numpy as np
import tensorflow as tf

# HuggingFace transformers 임포트 (없으면 안내 메시지)
try:
    import transformers
    from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
    print(f'transformers 버전: {transformers.__version__}')
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    print('transformers 라이브러리가 설치되지 않았습니다.')
    print('설치 방법: pip install transformers')
    print('이 노트북의 코드 셀은 transformers 설치 후 실행 가능합니다.')
    TRANSFORMERS_AVAILABLE = False

print(f'TensorFlow 버전: {tf.__version__}')

# TF 백엔드 사용 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # TF 경고 메시지 억제

## 2. BERT 사전학습 목표

BERT(Bidirectional Encoder Representations from Transformers)는 두 가지 자기지도학습(self-supervised) 목표로 사전학습된다.

### 2-1. MLM (Masked Language Model)

입력 토큰의 **15%를 무작위로 마스킹**하고, 마스킹된 토큰을 예측하는 태스크.

```
입력: "나는 [MASK] 밥을 먹었다"
목표: [MASK] → "오늘"
```

| 처리 방식 | 비율 | 설명 |
|-----------|------|------|
| `[MASK]` 교체 | 80% | 실제 마스킹 |
| 무작위 토큰 교체 | 10% | 노이즈 추가 |
| 원래 토큰 유지 | 10% | 복원 능력 학습 |

**GPT(단방향)**와 달리 BERT는 **양방향(Bidirectional)**으로 문맥을 봄 → 더 풍부한 표현 학습

### 2-2. NSP (Next Sentence Prediction)

두 문장 A, B가 **실제로 연속된 문장인지 예측**하는 이진 분류 태스크.

```
[CLS] 문장A [SEP] 문장B [SEP]
→ IsNext (50%) 또는 NotNext (50%)
```

- `[CLS]` 토큰의 최종 표현이 문장 쌍 분류에 사용됨
- 문서 이해, QA, NLI 등 문장 간 관계 파악 태스크에 유용

### 2-3. BERT 입력 형식

```
토큰:    [CLS]  나는  오늘  [SEP]  학교에  갔다  [SEP]
세그먼트: 0      0    0      0      1       1     1
위치:     0      1    2      3      4       5     6
```

## 3. HuggingFace + TF 백엔드 사용 방법

### 설치

```bash
pip install transformers
pip install tensorflow  # 이미 설치된 경우 생략
```

### TF 백엔드 강제 설정

```python
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
```

### HuggingFace 모델 네이밍 규칙

| 접두사 | 백엔드 | 예시 |
|--------|--------|------|
| `TF` + 모델명 | TensorFlow | `TFAutoModelForSequenceClassification` |
| 모델명 (접두사 없음) | PyTorch | `AutoModelForSequenceClassification` |

### Fine-Tuning 전략

1. **전체 파인튜닝 (Full Fine-Tuning)**: 모든 레이어를 업데이트 → 높은 성능, 느린 학습
2. **헤드만 학습 (Feature Extraction)**: BERT 가중치 동결, 분류 헤드만 학습 → 빠름, 낮은 성능
3. **점진적 파인튜닝 (Gradual Unfreezing)**: 위쪽 레이어부터 순차적으로 해동

In [ ]:
if TRANSFORMERS_AVAILABLE:
    # 사용할 사전학습 모델 지정
    # 영어: 'bert-base-uncased'
    # 한국어: 'klue/bert-base' (KLUE 벤치마크 기반)
    MODEL_NAME = 'bert-base-uncased'  # 영어 모델 (다운로드 크기 ~400MB)
    
    print(f'모델 로딩: {MODEL_NAME}')
    print('(처음 실행 시 HuggingFace Hub에서 모델을 다운로드합니다...)')
    
    # AutoTokenizer: 모델에 맞는 토크나이저 자동 선택
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # TFAutoModelForSequenceClassification: 이진/다중 분류용 모델 (TF 백엔드)
    # num_labels=2 → 긍정/부정 이진 분류
    bert_model = TFAutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )
    
    print(f'\n토크나이저 타입: {type(tokenizer).__name__}')
    print(f'모델 타입:       {type(bert_model).__name__}')
    print(f'어휘 크기:       {tokenizer.vocab_size}')
    print(f'최대 시퀀스 길이: {tokenizer.model_max_length}')
else:
    print('transformers가 설치되지 않아 이 셀을 건너뜁니다.')
    print('pip install transformers 후 재실행하세요.')

In [ ]:
if TRANSFORMERS_AVAILABLE:
    # 토크나이저 사용 예시
    sample_texts = [
        "This movie was absolutely fantastic! I loved every minute.",
        "Terrible film. Complete waste of time.",
        "An average movie with some good moments."
    ]
    
    print('=== 단일 문장 토크나이징 ===')
    single_enc = tokenizer(
        sample_texts[0],
        max_length=32,
        padding='max_length',
        truncation=True,
        return_tensors='tf'  # TensorFlow 텐서 반환
    )
    print(f'input_ids:      {single_enc["input_ids"][0].numpy()[:15]}...')
    print(f'attention_mask: {single_enc["attention_mask"][0].numpy()[:15]}...')
    print(f'shape: {single_enc["input_ids"].shape}')
    
    # 특수 토큰 확인
    print(f'\n[CLS] 토큰 ID: {tokenizer.cls_token_id}')
    print(f'[SEP] 토큰 ID: {tokenizer.sep_token_id}')
    print(f'[PAD] 토큰 ID: {tokenizer.pad_token_id}')
    print(f'[MASK] 토큰 ID: {tokenizer.mask_token_id}')
    
    print('\n=== 토큰 → 원문 복원 ===')
    token_ids = single_enc['input_ids'][0].numpy()
    tokens = tokenizer.convert_ids_to_tokens(token_ids[:12])
    print('토큰:', tokens)
    
    print('\n=== 배치 토크나이징 (패딩 자동 처리) ===')
    batch_enc = tokenizer(
        sample_texts,
        max_length=64,
        padding=True,         # 가장 긴 문장 길이에 맞춤
        truncation=True,
        return_tensors='tf'
    )
    print(f'input_ids shape:      {batch_enc["input_ids"].shape}')
    print(f'attention_mask shape: {batch_enc["attention_mask"].shape}')
    print('attention_mask (1=실제 토큰, 0=패딩):')
    print(batch_enc['attention_mask'].numpy())
else:
    print('transformers가 설치되지 않아 이 셀을 건너뜁니다.')

In [ ]:
if TRANSFORMERS_AVAILABLE:
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.rcParams['font.family'] = 'AppleGothic'
    matplotlib.rcParams['axes.unicode_minus'] = False

    # 간단한 감성 분류 데이터셋 (데모용)
    train_texts = [
        "I loved this movie, it was amazing!",
        "Absolutely wonderful film, highly recommend!",
        "Great acting and fantastic story.",
        "One of the best movies I have ever seen.",
        "Outstanding performance by all actors.",
        "Terrible movie, I hated every second.",
        "Worst film ever made, complete disaster.",
        "Boring and predictable, do not watch.",
        "Awful acting and terrible script.",
        "A complete waste of my time and money."
    ]
    train_labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]  # 1=긍정, 0=부정

    val_texts = [
        "Pretty good movie overall.",
        "Not impressed, expected better."
    ]
    val_labels = [1, 0]

    # 토크나이징
    MAX_LEN = 64

    def encode_texts(texts, max_len=MAX_LEN):
        return tokenizer(
            texts,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

    train_encodings = encode_texts(train_texts)
    val_encodings   = encode_texts(val_texts)

    # TF Dataset 생성
    def make_dataset(encodings, labels):
        dataset = tf.data.Dataset.from_tensor_slices((
            dict(encodings),
            labels
        ))
        return dataset.batch(4)

    train_dataset = make_dataset(train_encodings, train_labels)
    val_dataset   = make_dataset(val_encodings, val_labels)

    # Fine-Tuning 설정
    LEARNING_RATE = 2e-5  # BERT Fine-Tuning 권장 학습률
    EPOCHS = 3

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    bert_model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    print('BERT Fine-Tuning 시작...')
    print(f'학습 데이터: {len(train_texts)}개 | 검증 데이터: {len(val_texts)}개')
    print('(실제 사용 시 더 많은 데이터와 에포크가 필요합니다)')

    history = bert_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )

    # 추론 예시
    print('\n=== 추론 결과 ===')
    test_sentences = [
        "This was the best movie of the year!",
        "I fell asleep halfway through."
    ]

    test_enc = encode_texts(test_sentences)
    logits = bert_model.predict(dict(test_enc), verbose=0).logits
    probs = tf.nn.softmax(logits, axis=-1).numpy()

    for i, (sent, prob) in enumerate(zip(test_sentences, probs)):
        pred_label = '긍정' if prob[1] > prob[0] else '부정'
        print(f'문장: "{sent}"')
        print(f'  부정 확률: {prob[0]:.3f} | 긍정 확률: {prob[1]:.3f} → 예측: {pred_label}\n')
else:
    print('transformers가 설치되지 않아 이 셀을 건너뜁니다.')
    print('pip install transformers 후 재실행하세요.')

## 7. 한국어 BERT 모델 목록

### 주요 한국어 사전학습 모델 (HuggingFace Hub)

| 모델명 | 기관 | 특징 | 파라미터 수 |
|--------|------|------|------------|
| `klue/bert-base` | KLUE 팀 | KLUE 벤치마크 기반, 범용 | 110M |
| `klue/roberta-base` | KLUE 팀 | RoBERTa 아키텍처, 높은 성능 | 110M |
| `snunlp/KR-ELECTRA-discriminator` | SNU NLP | ELECTRA 기반, 빠른 학습 | 110M |
| `monologg/koelectra-base-v3-discriminator` | monologg | KoELECTRA v3 | 110M |
| `beomi/kcbert-base` | beomi | KcBERT (커뮤니티 댓글 학습) | 110M |
| `kakaobank/kf-deberta-base` | 카카오뱅크 | DeBERTa 기반 | 185M |
| `skt/kobert-base-v1` | SKT | KoBERT | 92M |

### 로드 방법

```python
# 한국어 BERT (KLUE) 로드 예시
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
model = TFAutoModelForSequenceClassification.from_pretrained(
    'klue/bert-base',
    num_labels=2
)
```

### KLUE 벤치마크 태스크

| 태스크 | 설명 | 모델 클래스 |
|--------|------|------------|
| TC (감성 분류) | 긍/부정 분류 | `TFAutoModelForSequenceClassification` |
| STS (문장 유사도) | 두 문장 유사도 점수 | `TFAutoModelForSequenceClassification` |
| NLI (자연어 추론) | 함의/중립/모순 분류 | `TFAutoModelForSequenceClassification` |
| NER (개체명 인식) | 인물/장소/날짜 태깅 | `TFAutoModelForTokenClassification` |
| QA (기계독해) | 지문에서 답변 추출 | `TFAutoModelForQuestionAnswering` |

## 8. 정리

### BERT Fine-Tuning 핵심 요약

1. **사전학습(Pre-training)**: 대규모 코퍼스에서 MLM + NSP로 언어 표현 학습
2. **파인튜닝(Fine-Tuning)**: 소량의 레이블 데이터로 특정 태스크에 적응
3. **토크나이저**: WordPiece/BPE로 서브워드 분리 → OOV 문제 해결
4. **특수 토큰**: `[CLS]`(문장 분류), `[SEP]`(문장 구분), `[PAD]`(패딩), `[MASK]`(마스킹)

### Fine-Tuning 권장 하이퍼파라미터

| 파라미터 | 권장 범위 |
|----------|---------|
| 학습률 | 2e-5 ~ 5e-5 |
| 에포크 수 | 3 ~ 5 |
| 배치 크기 | 16, 32 |
| 최대 시퀀스 길이 | 128, 256, 512 |
| Warm-up 비율 | 전체 스텝의 10% |

### 주의 사항

- 영어 BERT는 영어 데이터, 한국어 BERT는 한국어 데이터에 적합
- 도메인이 다를수록(예: 의료, 법률) 도메인 특화 모델 또는 추가 도메인 사전학습 권장
- GPU 메모리 부족 시: 배치 크기 감소, 시퀀스 길이 감소, 그래디언트 누적(gradient accumulation) 활용